# ZAF003_CV_FaceMaskDetection

## Overview

Face mask detection is an image processing and computer vision task that involves detecting whether a person in an image or video is wearing a face mask or not. With the ongoing COVID-19 pandemic, face masks have become a crucial tool in preventing the spread of the virus. Face mask detection systems can be used in public spaces such as airports, malls, and hospitals to ensure that individuals are following safety protocols.

## Methodology

- Import Libraries: First, you need to import the necessary libraries, including PyTorch, NumPy, OpenCV, and Matplotlib.

- Load the Data: Load the dataset of images with labeled classes (mask or no mask) and split it into training, validation, and testing sets. You can use PyTorch's DataLoader to load and preprocess the data.

- Load Pretrained Model: Load the pre-trained ResNet model from PyTorch's model zoo. ResNet is a popular choice for image classification tasks due to its superior performance on large datasets.

- Freeze Layers: Freeze the weights of the convolutional layers in the pre-trained model to prevent them from being updated during training.

- Replace the Classifier: Replace the final fully connected layer of the pre-trained model with a new classifier that has the appropriate number of output classes (2 for mask and no mask).

- Define Loss Function: Define the loss function for the binary classification problem, such as binary cross-entropy loss.

- Define Optimizer: Define the optimizer, such as stochastic gradient descent (SGD), to update the weights of the new classifier.

- Train the Model: Train the new classifier using the training set and validate it on the validation set. Use PyTorch's training loop to perform iterations of forward and backward propagation, and update the model's weights based on the loss and gradients.

- Test the Model: Test the final trained model on the testing set to evaluate its performance.

- Fine-Tune the Model: Fine-tune the model by unfreezing some of the earlier layers in the pre-trained model and continuing to train the model with a lower learning rate.

- Deploy the Model: Deploy the trained model for inference on new images or videos. Use the trained model to detect face masks in real-time scenarios.

## Business Segments
1. Healthcare
2. Safety

## Data
Face Mask Detection Datset - [Link](https://www.kaggle.com/datasets/omkargurav/face-mask-dataset)

## Papers
1. Resnet Architecture - [Link](https://arxiv.org/abs/1512.03385)

## Demo Link

## Importing Libraries

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from torchvision.datasets import ImageFolder
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import random_split
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import time
import copy

In [32]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Defining Global Variables

In [33]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = "/content/drive/MyDrive/Colab Notebooks/Image Data"

# Defining model name
model_name = "resnet"

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

## Defining Helper Functions

In [34]:
# Class to save best model
class SaveBestModel:
    """
    Class to save the best model while training. If the current epoch's 
    validation loss is less than the previous least less, then save the
    model state.
    """
    def __init__(
        self, best_valid_loss=float('inf')
    ):
        self.best_valid_loss = best_valid_loss
        
    def __call__(
        self, current_valid_loss, 
        epoch, model, optimizer, criterion
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")
            torch.save(model, '/content/drive/MyDrive/Colab Notebooks/Image Data/best_model.pth')

In [35]:
save_best_model = SaveBestModel()

In [36]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)
                save_best_model(
                epoch_loss, epoch, model, optimizer, criterion
            )

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [37]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

## Loading pre-trained model

In [38]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet34
        """
        model_ft = models.resnet34(weights=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

Invalid model name, exiting...
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

## Loading Dataset

In [39]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'with mask': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'without mask': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create datset
dataset = ImageFolder(data_dir, data_transforms['with mask'])

# Creating Train and validation dataset
val_pct = 0.1
val_size = int(val_pct * len(dataset))
train_ds, valid_ds = random_split(dataset, [len(dataset) - val_size, val_size])
train_dl = DataLoader(train_ds, batch_size, shuffle=True, num_workers=2)
valid_dl = DataLoader(valid_ds, batch_size, num_workers=2, shuffle=True)

dataloaders_dict = {'train': train_dl,
                    'val': valid_dl}


# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


## Model Training

In [40]:
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.0001, momentum=0.9)

Params to learn:
	 fc.weight
	 fc.bias


In [41]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, 
                             optimizer_ft, num_epochs=num_epochs, 
                             is_inception=False)

Epoch 0/14
----------
train Loss: 0.4700 Acc: 0.7950
val Loss: 0.3238 Acc: 0.8905

Best validation loss: 0.3238043616505435

Saving best model for epoch: 1


Epoch 1/14
----------
train Loss: 0.3354 Acc: 0.8838
val Loss: 0.2516 Acc: 0.9197

Best validation loss: 0.25164739711441264

Saving best model for epoch: 2


Epoch 2/14
----------
train Loss: 0.2978 Acc: 0.8886
val Loss: 0.1964 Acc: 0.9416

Best validation loss: 0.19638906343139872

Saving best model for epoch: 3


Epoch 3/14
----------
train Loss: 0.2695 Acc: 0.8862
val Loss: 0.1356 Acc: 0.9635

Best validation loss: 0.13564982591536792

Saving best model for epoch: 4


Epoch 4/14
----------
train Loss: 0.2397 Acc: 0.9104
val Loss: 0.1519 Acc: 0.9562

Epoch 5/14
----------
train Loss: 0.2542 Acc: 0.8959
val Loss: 0.1696 Acc: 0.9270

Epoch 6/14
----------
train Loss: 0.2423 Acc: 0.9031
val Loss: 0.1665 Acc: 0.9416

Epoch 7/14
----------
train Loss: 0.2181 Acc: 0.9177
val Loss: 0.1809 Acc: 0.9489

Epoch 8/14
----------
train Loss:

## Loading model

In [42]:
saved_model = torch.load("/content/drive/MyDrive/Colab Notebooks/Image Data/best_model.pth")